# Audio Retrieval MS2
This project focuses on audio retrieval using constellation maps, inspired by prior work on audio fingerprinting and matching (Please see the `README.md` in the Acknowledgements/Credits section for more information). Milestone 2 focuses on the following tasks: 
- Hash generation and storage
- Audio identification
- Scale-Up

We chose the best performing configurations as evaluated in Milestone 1.  

In [1]:
configuration = (8, 2) # (κ=8, τ=2)